In [1]:
from dotenv import load_dotenv
import os
import itertools
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
from broker import Broker
from backtester import Backtester
from tradetype import TradeType
from risk_management.riskmanager import RiskManager

In [3]:
from risk_management.stop_loss import ATRStopLossFinder, SimpleStopLossFinder, LowHighStopLossFinder
from risk_management.take_profit import RiskRewardTakeProfitFinder, SimpleTakeProfitFinder

In [4]:
from strategy.PSARZeroLagEMAStrategy import PSARZeroLagEMAStrategy
from strategy.EngulfingSMA import EngulfingSMA
from strategy.AwesomeOscillatorBBStrategy import AwesomeOscillatorBBStrategy
from strategy.BollingerEngulfing import BollingerEngulfing
from strategy.KangarooTailStrategy import KangarooTailStrategy
from strategy.SwingIndicatorStrategy import SwingIndicatorStrategy
from strategy.ExtremeEuphoriaBBStrategy import ExtremeEuphoriaBBStrategy
from strategy.StructuredDurationStrategy import StructuredDurationStrategy

In [5]:
load_dotenv()

API_KEY = os.getenv('API_KEY')
API_SECRET = os.getenv('API_SECRET')

broker = Broker(API_KEY, API_SECRET)

In [34]:
symbols = [
    'ETHUSDT',
    'XRPUSDT',
    'BTCUSDT',
    'SOLUSDT'
]
timeframes = [
    '1m',
    '3m',
    # '5m'
]
row_number = 5000
strategies = [
    # SwingIndicatorStrategy(),
    BollingerEngulfing(),
    # AwesomeOscillatorBBStrategy(),
    EngulfingSMA(),
    KangarooTailStrategy(),
    ExtremeEuphoriaBBStrategy(),
    # StructuredDurationStrategy()
    # PSARZeroLagEMAStrategy()
]

atr_stop_loss_finders = [ATRStopLossFinder(multiplier=multi) for multi in [0.88, 0.9, 1, 1.2, 1.5]]
highlow_loss_finders = [LowHighStopLossFinder(lookback_period=lookback) for lookback in [5, 10, 15, 20, 50, 100]]
stop_loss_finders = [SimpleStopLossFinder()] + atr_stop_loss_finders + highlow_loss_finders
take_profit_finders = [RiskRewardTakeProfitFinder(risk_reward_ratio=reword) for reword in [1, 1.5, 2, 3, 4]]
combinations = list(itertools.product(symbols, timeframes, strategies, stop_loss_finders, take_profit_finders))
results_list = []

In [35]:
for symbol, timeframe, strategy, stop_loss_finder, take_profit_finder in tqdm(combinations):
    ohlcv = broker.get_historical_data(symbol, timeframe, limit=row_number)
    stop_loss_finder.set_ohlcv(ohlcv)
    rms = [RiskManager(stop_loss_finder, take_profit_finder)]
    rm = RiskManager(stop_loss_finder, take_profit_finder)
    backtester = Backtester(strategy, rm, ohlcv)
    result = backtester.run(trade_type=TradeType.BOTH)
    result.update({
        'symbol': symbol,
        'timeframe': timeframe,
        'strategy': strategy,
        'stop_loss_finder': stop_loss_finder,
        'take_profit_finder': take_profit_finder,
        'use_trailing_stop_loss': rm.use_trailing_stop_loss,
        'win_rate': result['win_rate'] * 100
    })
    results_list.append(result)
    print(f"{symbol} orders for strategy: {strategy}, TimeFrame: {timeframe}, StopLossFinder: {stop_loss_finder}, TakeProfitFinder: {take_profit_finder}")
    print(backtester.get_orders().head())

  0%|          | 0/1920 [00:00<?, ?it/s]

ETHUSDT orders for strategy: BollingerEngulfing, TimeFrame: 1m, StopLossFinder: SimpleStopLossFinder(stop_loss_pct=0.02), TakeProfitFinder: RiskRewardTakeProfitFinder(risk_reward_ratio=1)
    side    entry     exit   profit  stop loss  take profit
0   long  1771.23  1812.34  11.5108    1735.81      1806.65
1  short  1836.89  1795.11  11.2806    1873.63      1800.15
2   long  1800.44  1757.61 -11.9924    1764.43      1836.45
3  short  1752.99  1803.91 -14.7668    1788.05      1717.93
4  short  1795.69  1800.52  -1.3041    1831.60      1759.78
ETHUSDT orders for strategy: BollingerEngulfing, TimeFrame: 1m, StopLossFinder: SimpleStopLossFinder(stop_loss_pct=0.02), TakeProfitFinder: RiskRewardTakeProfitFinder(risk_reward_ratio=1.5)
    side    entry     exit   profit  stop loss  take profit
0   long  1771.23  1836.89  18.3848    1735.81      1824.36
1  short  1821.40  1757.61  17.8612    1857.83      1766.76
2  short  1752.99  1803.91 -14.7668    1788.05      1700.40
3  short  1795.69  180

KeyboardInterrupt: 

In [24]:
results_df = pd.DataFrame(results_list)
sorded_results = results_df.sort_values(by="total_pnl", ascending=False)

In [25]:
sorded_results.head()

,total_trades,successful_trades,win_rate,rate_of_return,total_pnl,average_pnl,sharpe_ratio,max_consecutive_wins,max_consecutive_losses,max_drawdown,symbol,timeframe,strategy,stop_loss_finder,take_profit_finder,use_trailing_stop_loss
16,7,4,57.142857,0.156743,156.743027,22.391861,0.493549,2,1,0.016789,XRPUSDT,1m,EngulfingSMA(fast_sma_period=21),"ATRStopLossFinder(multiplier=0.9, atr_period=14)",RiskRewardTakeProfitFinder(risk_reward_ratio=1.5),False
47,1,1,100.000000,0.105229,105.228820,105.228820,NaN,1,0,0.000000,SOLUSDT,1m,ExtremeEuphoriaBBStrategy,"LowHighStopLossFinder(stop_loss_pct=0.02, look...",RiskRewardTakeProfitFinder(risk_reward_ratio=1.5),False
46,1,1,100.000000,0.104078,104.077520,104.077520,NaN,1,0,0.000000,SOLUSDT,1m,ExtremeEuphoriaBBStrategy,"ATRStopLossFinder(multiplier=0.9, atr_period=14)",RiskRewardTakeProfitFinder(risk_reward_ratio=1.5),False
7,1,1,100.000000,0.084996,84.996100,84.996100,NaN,1,0,0.000000,ETHUSDT,1m,"KangarooTailStrategy(lookback=200, sma_period=...","ATRStopLossFinder(multiplier=0.9, atr_period=14)",RiskRewardTakeProfitFinder(risk_reward_ratio=1.5),False
10,1,1,100.000000,0.082508,82.507600,82.507600,NaN,1,0,0.000000,ETHUSDT,1m,ExtremeEuphoriaBBStrategy,"ATRStopLossFinder(multiplier=0.9, atr_period=14)",RiskRewardTakeProfitFinder(risk_reward_ratio=1.5),False
